In [243]:
import os
from dotenv import load_dotenv, find_dotenv
env_path = find_dotenv()
load_dotenv(env_path)
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
GOOGLE_CX = os.getenv('GOOGLE_CX')

#from google_images_search import GoogleImagesSearch
#from google.cloud import vision

In [244]:
def strip(response):
    
    # remove these chars from entry
    chars_to_remove = '0123456789!"\'#$%&()*+,-./:;<=>?@[\]^_`{|}~♦●★‒…£¡™¢∞§¶•ªº–≠≠œ∑´®†¥¨≤≥÷ç√€'

    # remove these words from entry
    words_to_remove = ['serve','served','serving','appetizer','appetizers','course','price']

    # remove entry if it contains any of these
    drop_contain_words = ['menu','tax','consumer','advisory']

    # remove entry if it exactly matches any of these
    drop_exact_words = ['thank you', 'ipsum lorum']
    
    
    text = response.text_annotations[0].description
    menu_original = text.split('\n')
    
    menu_chars_removed = []
    for item in menu_original:
        for char in chars_to_remove:
            item = item.replace(char,'')
        menu_chars_removed.append(item)
        
    menu_words_removed = []
    for item in menu_chars_removed:
        temporary = []
        for word in item.split(' '):
            if word.lower() not in words_to_remove:
                temporary.append(word)
        remaining_words = ' '.join(temporary)
        menu_words_removed.append(remaining_words)
        
    menu_contains_dropped = []
    for item in menu_words_removed:
        temporary = []
        for word in item.split(' '):
            if word.lower() in drop_contain_words:
                temporary = []
                pass
            else:
                temporary.append(word)
        remaining_words = ' '.join(temporary)
        menu_contains_dropped.append(remaining_words)
    
    menu_exact_matches_dropped = []
    for item in menu_contains_dropped:
        if item.lower() in drop_exact_words:
            pass
        else:
            menu_exact_matches_dropped.append(item)
    
    too_short_dropped = [item for item in menu_exact_matches_dropped if len(item)>3]
    stripped_menu = too_short_dropped
   
    print(menu_original)
    return(stripped_menu)

In [285]:
def optimized_image_fetch_and_check(query):
    from google_images_search import GoogleImagesSearch
    from google.cloud import vision
    
    print(f'searching for {query}...')
    print()

    gis = GoogleImagesSearch(GOOGLE_API_KEY,GOOGLE_CX)
    
    _search_params = {
    'q': f'{query} recipe',
    'num': 1,
    'imgSize': 'large',
    'imgType': 'photo',
    'imgColorType': 'color'}
    

    gis.search(search_params=_search_params)
    print('fetching image:')
    if len(gis.results()) == 0:
        print('no image found, not verified as food.')
        print()
        return None
    
    url = gis.results()[0].url
    print(url)
    print()
    
    verified_queries = ['cheeseburger','pizza','fried chicken','ice cream sundae']
    
    if query.lower() in verified_queries:
        print(f'{query} already in known foods database, no need to verify!')
        print()
        return url
    
    client = vision.ImageAnnotatorClient()
    image = vision.Image()
    image.source.image_uri = url
    
    response = client.label_detection(image=image, max_results=1)
    label = [lab.description for lab in response.label_annotations]
    score = [lab.score for lab in response.label_annotations]
    
    print('verification filter:')
    print('label must be Food')
    print('score must be above .96')
    print()
    print(f'label: {label}')
    print(f'label score: {score}')
    print()
    
    if label[0] == 'Food' and score[0] > .96:
        print('verified as food!')
        print()
        print(url)
        print()
        return url
    
    if label[0] in ['Food', 'Tableware']:
        text_response = client.text_detection(image=image)
        texts = text_response.text_annotations
        n_chars = 0
        if len(texts)>0:
            n_chars = len(texts[0].description)
    
    
        print('verification filter:')
        print('label must be Food or Tableware')
        print('score must be above .96')
        print('number of chars must be below 100')
        print()
    
        print(f'label: {label}')
        print(f'label score: {score}')
        print(f'chars detected: {n_chars}')
        print()
    
        if (label[0] == 'Food' or label[0] == 'Tableware') and score[0] > .95 and n_chars < 100:
            print('verified as food!')
            print()
            print(url)
            print()
            return url
    
    
    _search_params = {
    'q': f'{query} recipe',
    'num': 3,
    'imgSize': 'large',
    'imgType': 'photo',
    'imgColorType': 'color',
    'safe': 'medium'}
        
    
    gis = GoogleImagesSearch(GOOGLE_API_KEY,GOOGLE_CX)
    gis.search(search_params=_search_params)
    urls = [result.url for result in gis.results()]
    urls = urls[1:]
    print('fetching additional images:')
    for url in urls:
        print(url)
    print()
    
    print('verification filter:')
    print('label must be Food')
    print('score must be above .96')
    print()
    
    labels = []
    scores = []
    for url in urls:
        image.source.image_uri = url
        response = client.label_detection(image=image, max_results=1)
        label = [lab.description for lab in response.label_annotations]
        score = [lab.score for lab in response.label_annotations]
        labels.append(label)
        scores.append(score)
        
    print(f'labels: {labels}')
    print(f'label scores: {scores}')
    print()
    
    for label,score in zip(labels,scores):
        if label[0] == 'Food' and score[0] > .96:
            print('verified as food!')
            print()
            print(urls[labels.index(label)])
            print()
            return urls[labels.index(label)]
        
    if label[0] in ['Food', 'Tableware']:
        char_counts = []   
        for url in urls:
            text_response = client.text_detection(image=image)
            texts = text_response.text_annotations
            n_chars = 0
            if len(texts)>0:
                n_chars = len(texts[0].description)
            char_counts.append(n_chars)
        
        print('verification filter:')
        print('label must be Food or Tableware')
        print('score must be above .95')
        print('number of chars must be below 100')
        print()  
    
        print(f'labels: {labels}')
        print(f'label score: {scores}')
        print(f'chars detected: {char_counts}')
        print()
    
        for label,score,chars in zip(labels,scores,char_counts):
            if (label[0] == 'Food' or label[0] == 'Tableware') and score[0] > .95 and chars < 100:
                print('verified as food!')
                print()
                print(urls[labels.index(label)])
                print()
                return urls[labels.index(label)]
        
    print('not verified as food.')
    print()
    return None

In [257]:
%%time
url = optimized_image_fetch_and_check('asodviaoivn')

searching for asodviaoivn...

fetching image:
no image found, not verified as food.

CPU times: user 20 ms, sys: 3.19 ms, total: 23.2 ms
Wall time: 763 ms


In [258]:
%%time
url = optimized_image_fetch_and_check('pizza')

searching for pizza...

fetching image:
https://www.indianhealthyrecipes.com/wp-content/uploads/2015/10/pizza-recipe-1-500x500.jpg

pizza already in known foods database, no need to verify!
https://www.indianhealthyrecipes.com/wp-content/uploads/2015/10/pizza-recipe-1-500x500.jpg

CPU times: user 44.1 ms, sys: 5.27 ms, total: 49.4 ms
Wall time: 1.9 s


In [280]:
%%time
url = optimized_image_fetch_and_check('ceasar salad')

searching for ceasar salad...

fetching image:
https://natashaskitchen.com/wp-content/uploads/2019/01/Caesar-Salad-Recipe-3-500x375.jpg

verification filter:
label must be Food
score must be above .96

label: ['Food']
label score: [0.9869239926338196]

verified as food!

https://natashaskitchen.com/wp-content/uploads/2019/01/Caesar-Salad-Recipe-3-500x375.jpg

CPU times: user 113 ms, sys: 94.2 ms, total: 208 ms
Wall time: 3.27 s


In [286]:
%%time
optimized_image_fetch_and_check('ice tea')

searching for ice tea...

fetching image:
https://s3.amazonaws.com/finecooking.s3.tauntonclud.com/app/uploads/2017/04/18144831/fc52th054-01-main.jpg

verification filter:
label must be Food
score must be above .96

label: ['Tableware']
label score: [0.9682804942131042]

verification filter:
label must be Food or Tableware
score must be above .96
number of chars must be below 100

label: ['Tableware']
label score: [0.9682804942131042]
chars detected: 0

verified as food!

https://s3.amazonaws.com/finecooking.s3.tauntonclud.com/app/uploads/2017/04/18144831/fc52th054-01-main.jpg

CPU times: user 132 ms, sys: 143 ms, total: 275 ms
Wall time: 6.2 s


'https://s3.amazonaws.com/finecooking.s3.tauntonclud.com/app/uploads/2017/04/18144831/fc52th054-01-main.jpg'

In [287]:
%%time
optimized_image_fetch_and_check('fresh juice')

searching for fresh juice...

fetching image:
https://i.pinimg.com/originals/83/92/75/8392751129549b2a08c1e524489c0a6a.jpg

verification filter:
label must be Food
score must be above .96

label: ['Tableware']
label score: [0.9718490242958069]

verification filter:
label must be Food or Tableware
score must be above .96
number of chars must be below 100

label: ['Tableware']
label score: [0.9718490242958069]
chars detected: 771

fetching additional images:
https://www.masalaherb.com/wp-content/uploads/2019/05/Passion-Fruit-Juice-gog-2-500x500.jpg
https://cdn2.foodviva.com/static-content/food-images/juice-recipes/watermelon-juice-recipe/watermelon-juice-recipe.jpg

verification filter:
label must be Food
score must be above .96

labels: [['Food'], ['Tableware']]
label scores: [[0.9828023314476013], [0.9636179804801941]]

verified as food!

https://www.masalaherb.com/wp-content/uploads/2019/05/Passion-Fruit-Juice-gog-2-500x500.jpg

CPU times: user 252 ms, sys: 234 ms, total: 486 ms
Wall 

'https://www.masalaherb.com/wp-content/uploads/2019/05/Passion-Fruit-Juice-gog-2-500x500.jpg'

In [288]:
%%time
optimized_image_fetch_and_check('caution wet floor')

searching for caution wet floor...

fetching image:
https://www.webstaurantstore.com/images/products/large/584422/2171608.jpg

verification filter:
label must be Food
score must be above .96

label: ['Azure']
label score: [0.8927730321884155]

fetching additional images:
https://cdnimg.webstaurantstore.com/images/products/large/608130/2242050.jpg
https://www.webstaurantstore.com/images/products/large/354215/1353214.jpg

verification filter:
label must be Food
score must be above .96

labels: [['Yellow'], ['Road surface']]
label scores: [[0.8636167049407959], [0.8598910570144653]]

not verified as food.

CPU times: user 245 ms, sys: 232 ms, total: 477 ms
Wall time: 9.63 s


In [289]:
%%time
optimized_image_fetch_and_check('iphone')

searching for iphone...

fetching image:
https://www.gymondo.com/magazin/wp-content/uploads/header_EN-620x413.jpg

verification filter:
label must be Food
score must be above .96

label: ['Food']
label score: [0.9447255730628967]

verification filter:
label must be Food or Tableware
score must be above .96
number of chars must be below 100

label: ['Food']
label score: [0.9447255730628967]
chars detected: 114

fetching additional images:
https://www.wikihow.com/images/thumb/b/bf/Add-Switch-Control-Recipes-on-an-iPhone-Step-8.jpg/v4-460px-Add-Switch-Control-Recipes-on-an-iPhone-Step-8.jpg.webp
https://fs.npstatic.com/userfiles/7687254/image/Apple_iPhone_13/NextPit_Apple_iPhone_13_test-w810h462.jpg

verification filter:
label must be Food
score must be above .96

labels: [['Communication Device'], ['Hand']]
label scores: [[0.8753791451454163], [0.9616590142250061]]

not verified as food.

CPU times: user 275 ms, sys: 310 ms, total: 584 ms
Wall time: 14 s


In [292]:
%%time
optimized_image_fetch_and_check('bintang')

searching for bintang...

fetching image:
https://media-cdn.tripadvisor.com/media/photo-s/0e/e0/13/73/secret-recipe-bintang.jpg

verification filter:
label must be Food
score must be above .96

label: ['Property']
label score: [0.9429903030395508]

fetching additional images:
https://media-cdn.tripadvisor.com/media/photo-s/0b/d5/a5/9e/decor.jpg
https://www.theyumlist.net/wp-content/uploads/2021/04/Ultime-Atelier-_-Boulangerie-Yum-List-LS-15-750x500.jpg

verification filter:
label must be Food
score must be above .96

labels: [['Furniture'], ['Food']]
label scores: [[0.9579998254776001], [0.9839783906936646]]

verified as food!

https://www.theyumlist.net/wp-content/uploads/2021/04/Ultime-Atelier-_-Boulangerie-Yum-List-LS-15-750x500.jpg

CPU times: user 265 ms, sys: 227 ms, total: 491 ms
Wall time: 8.35 s


'https://www.theyumlist.net/wp-content/uploads/2021/04/Ultime-Atelier-_-Boulangerie-Yum-List-LS-15-750x500.jpg'

In [293]:
%%time
optimized_image_fetch_and_check('bintang beer')

searching for bintang beer...

fetching image:
https://media-cdn.tripadvisor.com/media/photo-s/19/ce/45/a7/all-our-wooden-fired.jpg

verification filter:
label must be Food
score must be above .96

label: ['Food']
label score: [0.9779905676841736]

verified as food!

https://media-cdn.tripadvisor.com/media/photo-s/19/ce/45/a7/all-our-wooden-fired.jpg

CPU times: user 103 ms, sys: 60.4 ms, total: 163 ms
Wall time: 3.1 s


'https://media-cdn.tripadvisor.com/media/photo-s/19/ce/45/a7/all-our-wooden-fired.jpg'

In [104]:
path_start = '../raw_data/all_menus/'

paths = ['warung_sika_food.jpg',
'chinese_menu_1.png',
'english_menu_1.png',
'english_menu_2.jpg',
'english_menu_3.jpg',
'english_menu_4.jpg',
'english_menu_5.jpg',
'english_menu_6.jpg',
'english_menu_7.png',
'english_menu_8.png',
'english_menu_9.jpg',
'english_menu_10.jpg',
'english_menu_11.jpg',
'english_menu_12.png',
'indo_menu_1.png',
'saudi_menu_1.jpeg',
'spanish_menu_1.jpg',
'spanish_menu_2.jpg',
'thai_menu_1.png',
'warung_sika_drinks.jpg']

In [119]:
detected_menus = []
for path in paths:
    detected_menus.append(text_detection_full_response(path_start + path))

In [295]:
stripped_menus = []
for menu in detected_menus:
    stripped_menus.append(strip(menu))

['SWEET&SOUR CHICKEN 50', 'CHICKEN SATAY 35', 'NASI GORENG 25', 'MIE GORENG 25', 'CAP CAY 25', 'FUYUNG HAI 20', 'CHICKEN SOUP 30', 'NOODLE SOUP 20', 'GADO-GADO', '25', 'NASI', 'CAMPUR', 'sen', 'S', 'W', 'Fr']
['休闲至尊堡', '招牌咖喱 力', '♦咖喱鸡扒/猪扒饭', '咖喱牛肉饭', '●咖喱牛腩饭', '咖喱肥牛饭', "'咖喱海鲜饭", '咖喱滑鸡饭', '榨菜肉丝汤面', '鲜虾云吞', '●番茄肥牛汤河粉', '香辣牛肉面方', '雪菜肉丝面', '咖喱海鲜面丿', '酱香牛腩面', '鲜虾云吞面', '肠仔/火腿/烟肉煎蛋', '以上可配米粉/河粉/公仔面/意粉', '改乌冬面/车仔面加收3元', '粥类', '瑶柱鸡皇粥', '冬菇滑鸡粥', '皮蛋瘦肉粥', '窝蛋牛肉粥', '明火白粥', '人气出品', '新品推出', '需要时长约25分钟,敬请留意!', '带有辛辣,敬请留意!', '烫粉面类', '28', '35', '35', '30', '32', '30', '18', '23', '20', '20', '15', '20', '23', '23', '18', '15', '13', '13', '15', 'LO', '5', '5335', '至尊空食', '中式饭类', '意', '★台式卤肉饭', '25', '●经典意大利', '至尊鲍鱼鸡饭', '32', '米兰辣i', '★ 葡式咸猪手饭', '28', '三杯鸡饭', '28', '咖喱海鲜', '水煮肥牛饭', '32', '鲜茄牛柳', '白果支竹猪肚饭', '28', '26', '卡邦尼灯', '金菇肥牛饭', '番茄牛肉饭', '法式鸡皇饭', '28', '忌廉蛋汁', '25', '20', '拿破仑', '鱼香茄子饭', '韭黄滑蛋饭', '23', '加5元热', '虾仁滑蛋饭', '28', '叉烧滑蛋饭', '23', '生炒排骨饭', '28', '姜葱白切鸡饭', '30', '手撕鸡饭', '30', '蜜汁叉烧饭', '26

In [296]:
stripped_menus

[['SWEETSOUR CHICKEN ',
  'CHICKEN SATAY ',
  'NASI GORENG ',
  'MIE GORENG ',
  'CAP CAY ',
  'FUYUNG HAI ',
  'CHICKEN SOUP ',
  'NOODLE SOUP ',
  'GADOGADO',
  'NASI',
  'CAMPUR'],
 ['休闲至尊堡',
  '招牌咖喱 力',
  '咖喱鸡扒猪扒饭',
  '咖喱牛肉饭',
  '咖喱牛腩饭',
  '咖喱肥牛饭',
  '咖喱海鲜饭',
  '咖喱滑鸡饭',
  '榨菜肉丝汤面',
  '鲜虾云吞',
  '番茄肥牛汤河粉',
  '香辣牛肉面方',
  '雪菜肉丝面',
  '咖喱海鲜面丿',
  '酱香牛腩面',
  '鲜虾云吞面',
  '肠仔火腿烟肉煎蛋',
  '以上可配米粉河粉公仔面意粉',
  '改乌冬面车仔面加收元',
  '瑶柱鸡皇粥',
  '冬菇滑鸡粥',
  '皮蛋瘦肉粥',
  '窝蛋牛肉粥',
  '明火白粥',
  '人气出品',
  '新品推出',
  '需要时长约分钟敬请留意',
  '带有辛辣敬请留意',
  '烫粉面类',
  '至尊空食',
  '中式饭类',
  '台式卤肉饭',
  '经典意大利',
  '至尊鲍鱼鸡饭',
  '米兰辣i',
  ' 葡式咸猪手饭',
  '三杯鸡饭',
  '咖喱海鲜',
  '水煮肥牛饭',
  '鲜茄牛柳',
  '白果支竹猪肚饭',
  '卡邦尼灯',
  '金菇肥牛饭',
  '番茄牛肉饭',
  '法式鸡皇饭',
  '忌廉蛋汁',
  '鱼香茄子饭',
  '韭黄滑蛋饭',
  '虾仁滑蛋饭',
  '叉烧滑蛋饭',
  '生炒排骨饭',
  '姜葱白切鸡饭',
  '手撕鸡饭',
  '蜜汁叉烧饭',
  '香草茄汁鸡扒鸡扒饭 ',
  '鲜菇牛肉排骨饭',
  '凉瓜牛肉肉片饭',
  '萝卜牛腩饭',
  '支竹牛腩饭',
  '酱爆猪颈肉饭',
  '麻婆豆腐饭 炒',
  '以上出品送老火汤元配炖汤',
  '加元送热饮杯加元送冻饮杯',
  '咖喱海鲜蛋包饭',
  '虾仁瑶柱蛋包饭',
  '日式鳗鱼蛋包饭',
  '温馨提示',
  '、净饮每位收费元',
  '、顾客如在下

In [159]:
test_menu =  ['HOT ICE TEA ',
  'HOTICE JERUK ',
  'ICE LEMON TEA ',
  'BALI COFFEE ',
  'ICE COFFEE',
  'FRESH JUICE ',
  'HOT GINGER TEA ',
  'FRESH FRUITS ',
  'SOFTDRINKS',
  'MILKSHAKE KS ',
  'BINTANG',
  'BALT SIP ']

In [160]:
for item in test_menu:
    optimized_image_fetch_and_check(item)
    print('--------------------------------')

searching for HOT ICE TEA ...

fetching one image:
https://c.ndtvimg.com/2020-04/5cq314g8_green-iced-tea_625x300_10_April_20.jpg

[0.9582850337028503]
verify image is food...

image label:
['Tableware']

not verified as food.

fetching additional images:
https://hips.hearstapps.com/wdy.h-cdn.co/assets/cm/15/09/54ef91ca46f35_-_creamy-sweet-tea-w7jkbx-xl.jpg?crop=1xw:1.0xh;center,top&resize=480:*
https://food.fnr.sndimg.com/content/dam/images/food/fullset/2011/6/3/1/FNM_070111-Centerfold-007_s4x3.jpg.rend.hgtvcom.616.462.suffix/1371597847872.jpeg

[['Tableware'], ['Liquid']]

--------------------------------
searching for HOTICE JERUK ...

fetching one image:
https://www.thespruceeats.com/thmb/BSVtkEWwdF8P5UlMV27y9KiKZYk=/600x400/filters:max_bytes(150000):strip_icc()/pork_vindaloo-90226529-58f3bce65f9b582c4da610a7.jpg

[0.9836804866790771]
verify image is food...

image label:
['Food']

verified as food!

--------------------------------
searching for ICE LEMON TEA ...

fetching one imag

IndexError: list index out of range

In [221]:
optimized_image_fetch_and_check('hot ice tea')

searching for hot ice tea...

fetching one image:
https://c.ndtvimg.com/2020-04/5cq314g8_green-iced-tea_625x300_10_April_20.jpg

verify image is food...

image label:
['Tableware']
[0.9582850337028503]
46 characters detected

verified as food!



'https://c.ndtvimg.com/2020-04/5cq314g8_green-iced-tea_625x300_10_April_20.jpg'

In [300]:
for item in ['SWEETSOUR CHICKEN ',
  'CHICKEN SATAY ',
  'NASI GORENG ',
  'MIE GORENG ',
  'CAP CAY ',
  'FUYUNG HAI ',
  'CHICKEN SOUP ',
  'NOODLE SOUP ',
  'GADOGADO',
  'NASI',
  'CAMPUR']:
    optimized_image_fetch_and_check(item)
    
    

searching for SWEETSOUR CHICKEN ...

fetching image:
https://www.averiecooks.com/wp-content/uploads/2017/01/sweetsourchicken-10-720x540.jpg

verification filter:
label must be Food
score must be above .96

label: ['Food']
label score: [0.9869999289512634]

verified as food!

https://www.averiecooks.com/wp-content/uploads/2017/01/sweetsourchicken-10-720x540.jpg

searching for CHICKEN SATAY ...

fetching image:
https://rasamalaysia.com/wp-content/uploads/2019/04/chicken-satay-thumb.jpg

verification filter:
label must be Food
score must be above .96

label: ['Food']
label score: [0.9808286428451538]

verified as food!

https://rasamalaysia.com/wp-content/uploads/2019/04/chicken-satay-thumb.jpg

searching for NASI GORENG ...

fetching image:
https://rasamalaysia.com/wp-content/uploads/2009/09/nasi_goreng2.jpg

verification filter:
label must be Food
score must be above .96

label: ['Food']
label score: [0.9873111248016357]

verified as food!

https://rasamalaysia.com/wp-content/uploads/20

IndexError: list index out of range

In [303]:
optimized_image_fetch_and_check('FUYUNG HAI')

searching for FUYUNG HAI...

fetching image:
https://asianfoodjourney.de/wp-content/uploads/2020/07/FY2-500x500.png

verification filter:
label must be Food
score must be above .96

label: ['Food']
label score: [0.9842996001243591]

verified as food!

https://asianfoodjourney.de/wp-content/uploads/2020/07/FY2-500x500.png



'https://asianfoodjourney.de/wp-content/uploads/2020/07/FY2-500x500.png'

In [306]:
for item in ['GADOGADO',
  'NASI',
  'CAMPUR']:
    optimized_image_fetch_and_check(item)


searching for GADOGADO...

fetching image:
https://static01.nyt.com/images/2020/08/21/dining/hm-gado-gado/merlin_175613193_0c311354-1fdc-4b7b-ab23-4bd54c83df01-articleLarge.jpg

verification filter:
label must be Food
score must be above .96

label: ['Food']
label score: [0.9865169525146484]

verified as food!

https://static01.nyt.com/images/2020/08/21/dining/hm-gado-gado/merlin_175613193_0c311354-1fdc-4b7b-ab23-4bd54c83df01-articleLarge.jpg

searching for NASI...

fetching image:
https://static01.nyt.com/images/2022/01/26/dining/21SINGAPOREREX4-nasi-biryani/merlin_200287473_78778039-dc1b-4644-9f5c-818d61d24403-articleLarge.jpg

verification filter:
label must be Food
score must be above .96

label: ['Food']
label score: [0.9875624179840088]

verified as food!

https://static01.nyt.com/images/2022/01/26/dining/21SINGAPOREREX4-nasi-biryani/merlin_200287473_78778039-dc1b-4644-9f5c-818d61d24403-articleLarge.jpg

searching for CAMPUR...

fetching image:
https://img-global.cpcdn.com/recipe